In [ ]:
import asyncio
import nest_asyncio
import os, sys
import datetime
from collections import Counter

import warnings
nest_asyncio.apply()

warnings.simplefilter("error", RuntimeWarning)  # 경고를 예외로 변환

home_path = os.path.expanduser("~")
sys.path.append(os.path.join(home_path, "GitHub", "Thunder", "Binance"))

import Workspace.Utils.TradingUtils as tr_utils# import 
from Workspace.Services.PublicData.Receiver.FuturesMarketWebsocket import (
    FuturesMarketWebsocket as futures_mk_ws,
)
import Workspace.Utils.BaseUtils as base_utils
import SystemConfig
# import Workspace.Utils.BaseUtils as base_utils


symbols = SystemConfig.Streaming.symbols
stream_1 = "depth"
stream_2 = "trade"

ins_st1 = futures_mk_ws(symbols)
ins_st2 = futures_mk_ws(symbols)
async def main(minute:int):
    print(f"  ⏳ make the program wait")
    # await base_utils.sleep_next_minute(1)
    st1_result = []
    st2_result = []
    
    start_date = datetime.datetime.now()
    now_minute = int(start_date.minute)
    target_minute = int(start_date.minute) + minute
    await ins_st1.open_stream_connection(stream_1)
    print(f"  🔗 '{stream_1}' Websocket connect")
    await ins_st2.open_stream_connection(stream_2)
    print(f"  🔗 '{stream_2}' Websocket connect")
    i = 1
    print(f"  🚀 start receiving start")
    bids_prices = []
    asks_prices = []
    while now_minute < target_minute:
        # base_utils.std_print(f"        🏃🏃🏻‍♀️🏃🏽‍♂️💨 {i}회 진행")
        st1_message = await ins_st1.receive_message()
        st2_message = await ins_st2.receive_message()
        i +=1
        
        symbol_st1 = st1_message["data"]["s"]
        symbol_st2 = st2_message["data"]["s"]


        if symbol_st1 == "BTCUSDT":
            data = st1_message["data"]
            bids = data.get("b", [])  # 매수 주문 업데이트
            asks = data.get("a", [])  # 매도 주문 업데이트

            for bid in bids:
                price, quantity = bid
                if float(quantity) == 0:
                    convert_to_price = int(float(price))
                    bids_prices.append(convert_to_price)

            for ask in asks:
                price, quantity = ask
                if float(quantity) == 0:
                    convert_to_price = int(float(price))
                    asks_prices.append(convert_to_price)
    
        new_date = datetime.datetime.now()
        now_minute = int(new_date.minute)
        

    print(f"\n  ✅ 데이터 수신 완료")
    await ins_st1.close_connection()
    await ins_st2.close_connection()
    return bids_prices, asks_prices

message = asyncio.run(main(2))

bid, ask = message

data = ['apple', 'banana', 'apple', 'orange', 'banana', 'apple']
bid_count = Counter(bid)
bid_count.
bid_ranking = bid_count.most_common() 

ask_count = Counter(ask)
ask_ranking = ask_count.most_common() 

print(bid_ranking)
print(ask_ranking)

In [ ]:
ranking_asc = sorted(bid_ranking.items(), key=lambda x: x[1])

In [7]:
import nest_asyncio
nest_asyncio.apply()  # 주피터 노트북에서 asyncio 사용 가능하게 설정

import asyncio
import json
import websockets
from collections import deque

SYMBOL = "btcusdt"
URL = f"wss://stream.binance.com:9443/ws/{SYMBOL}@depth"

# 누적 델타 볼륨 계산을 위한 큐
delta_volume_queue = deque(maxlen=300)  # 최근 50개의 델타 값 저장

async def process_order_book():
    async with websockets.connect(URL) as ws:
        while True:
            data = await ws.recv()
            order_book = json.loads(data)
            
            bids = order_book["b"][:5]  # 상위 5개 매수 호가
            asks = order_book["a"][:5]  # 상위 5개 매도 호가

            # 최우선 매수(Bid) 및 매도(Ask) 가격
            best_bid = float(bids[0][0])
            best_ask = float(asks[0][0])
            spread = best_ask - best_bid  # 스프레드 계산

            # 매수·매도 물량 총합 (상위 5개 호가)
            total_bid_volume = sum(float(bid[1]) for bid in bids)
            total_ask_volume = sum(float(ask[1]) for ask in asks)

            # 오더북 불균형 계산
            imbalance = (total_bid_volume - total_ask_volume) / (total_bid_volume + total_ask_volume)

            # 대량 주문 감지 (100 BTC 이상 주문 존재 여부 확인)
            large_bid_orders = [bid for bid in bids if float(bid[1]) > 100]
            large_ask_orders = [ask for ask in asks if float(ask[1]) > 100]

            large_bid_detected = len(large_bid_orders) > 0
            large_ask_detected = len(large_ask_orders) > 0

            # 누적 델타 볼륨 업데이트
            delta_volume = total_bid_volume - total_ask_volume
            delta_volume_queue.append(delta_volume)
            avg_delta_volume = sum(delta_volume_queue) / len(delta_volume_queue)

            # 결과 출력
            print(f"\n🔹 Best Bid: {best_bid}, Best Ask: {best_ask}, Spread: {spread:.5f}")
            print(f"📊 Order Book Imbalance: {imbalance:.3f}")
            print(f"📈 Large Buy Orders Detected: {large_bid_detected}, 📉 Large Sell Orders Detected: {large_ask_detected}")
            print(f"📊 Cumulative Delta Volume (Last 50): {avg_delta_volume:.2f}")

            # 0.5초 대기 후 반복
            await asyncio.sleep(0.5)

asyncio.run(process_order_book())



🔹 Best Bid: 86202.27, Best Ask: 86202.28, Spread: 0.01000
📊 Order Book Imbalance: -0.891
📈 Large Buy Orders Detected: False, 📉 Large Sell Orders Detected: False
📊 Cumulative Delta Volume (Last 50): -7.44

🔹 Best Bid: 86202.27, Best Ask: 86202.28, Spread: 0.01000
📊 Order Book Imbalance: -0.866
📈 Large Buy Orders Detected: False, 📉 Large Sell Orders Detected: False
📊 Cumulative Delta Volume (Last 50): -6.67

🔹 Best Bid: 86202.27, Best Ask: 86202.28, Spread: 0.01000
📊 Order Book Imbalance: -0.849
📈 Large Buy Orders Detected: False, 📉 Large Sell Orders Detected: False
📊 Cumulative Delta Volume (Last 50): -6.21

🔹 Best Bid: 86202.27, Best Ask: 86202.28, Spread: 0.01000
📊 Order Book Imbalance: -0.379
📈 Large Buy Orders Detected: False, 📉 Large Sell Orders Detected: False
📊 Cumulative Delta Volume (Last 50): -5.21

🔹 Best Bid: 86202.27, Best Ask: 86205.99, Spread: 3.72000
📊 Order Book Imbalance: 0.995
📈 Large Buy Orders Detected: False, 📉 Large Sell Orders Detected: False
📊 Cumulative Delta 

ZeroDivisionError: float division by zero